# System on Chip design auxiliaries

For mass generation of bus decoders, a register bit map must be associated with corresponding control, status or data signals.
This SoC concept follows the [*MaSoCist*](https://github.com/hackfin/MaSoCist) register map design rules:

* Registers are mapped into memory space and are accessed by an address, hence.
* They can be flagged read-only, write-only or volatile:
  * READONLY: Writing to the register has no effect
  * WRITEONLY: Reading from this register returns an undefined value
  * VOLATILE: Write or read access triggers a pulse on the corresponding `select` lines.
    This allows to implement `W1C` (write one to clear) behaviour, or optimized data in/out transfers.
* Registers contain bit fields that can be READONLY or WRITEONLY
* Two register definitions (one READONLY, one WRITEONLY) can be mapped to one address. This is used for data I/O.

We import all needed MMR elements as follows:

In [3]:
from cyrite.library.soc import *

## Register definitions

Add a few register with bit fields and flags (these are normally taken from an external module or generated from an XML description).

This MMR scheme follows the register design scheme used by `gensoc` from the [MaSoCist SoC builder](https://github.com/hackfin/MaSoCist). Detailed information on how to interpret the register maps is found in the MaSoCist documentation.

In [4]:
reg01 = Reg(16,
    [
        BF("im", 3, 1, flags = BF.READONLY),
        BF("ex", 7, 6),
        BF("inv", 4, 4, flags = BF.WRITEONLY),
        BF("mode", 14, 10, default = 2)
    ]
)

reg02 = Reg(16,
    [
        BF("gna", 6, 1, default = 8),
        BF("reset", 7, 7, default = True)
    ],
    flags = Reg.VOLATILE | Reg.WRITEONLY
)

**Note** `VOLATILE` flagged registers create a special output pin in the `.select` container of the RegisterSignal which is pulsed when an access was made. See wave trace below.

## The MMR factory

The code generation for the memory mapped registers is done by the `MMRGenerator` class. We derive from it and define a register map.

In [5]:
class MMRDesign(MMRGenerator):
    # This is a description for an address map
    regdesc = {
        0x01: ['stat', reg01],
        0x02: ['ctrl', reg02],
        0x04: ['TXD',  Reg(16, [ BF("DATA", 15, 0)], flags = Reg.WRITEONLY | Reg.VOLATILE) ],
        0x05: ['RXD',  Reg(16, [ BF("DATA", 15, 0)], flags = Reg.READONLY | Reg.VOLATILE)]
    }

    def build(self, p : MMRPort, interface : dict):
        "Creates an instance of a MMR decoder"
        inst = self.mmr_decode(
            clk = p.clk,
            reset = p.rst,
            addr = p.addr,
            wr = p.wr,
            data_in = p.din,
            data_out = p.dout,
            REGDESC = self.regdesc,
            **interface
        )

        return inst

## The test bench simulation class

Now we derive from this class again in order to create a custom simulation class that includes a test bench.
We will be using a co-simulation environment, where the test bench code below is run in native Python while the instanced MMR decoder runs as compiled hardware in the background.

The test bench makes use of the MMRPort container, which supplies a few simulation macros to mimic write and read sequences. You can derive from this container to define own sequences.

In [6]:
from cyhdl import *

class SimMMR(MMRDesign):
    @cyrite_factory.testbench("ns")
    def testbench(self):
        p = MMRPort(self)

        clk = p.clk

        interface = self.generate_interface(self.regdesc)
        ctrl = interface['ctrl']
        stat = interface['stat']

        # This debug signal is not connected to the
        # simulation back end
        debug = self.Signal(bool(), name = 'debug')
        
        inst = self.build(p, interface)

        @self.always(delay(2))
        def clkgen():
            clk.next = ~clk

        @self.sequence
        def main():
            p.rst.next = True
            stat.read.ex.next = 0

            yield delay(10)

            yield clk.posedge
            print("START")

            debug.next = False
            p.wr.next = False
            p.addr.next = 0x001
     
            yield from p.reset_sequence()

            print("DONE RESET")

            assert p.rst == False
                    
            yield clk.negedge
            print("SETTING stat.read")
            stat.read.ex.next = 0
            stat.read.mode.next = 4
            stat.read.im.next = 2

            yield clk.negedge
            yield clk.negedge

            p.assert_read(0x001, 0x1004)
            yield clk.negedge
            
            yield from p.write_sequence(0x002, 0xfa)
            debug.next = True
            
            yield clk.negedge
            print("VAL", ctrl.select.sel_w)
            # assert ctrl.select.sel_w == True
            assert ctrl.write.gna == 0x3d
            yield clk.posedge
            yield clk.negedge
            assert ctrl.select.sel_w == False

            yield from p.write_sequence(0x001, 0x10)
            yield clk.negedge # Here, we can see different
            # behaviour of icarus vs ghdl. Icarus needs this clk.negedge to
            # update its signals, GHDL doesn't
            assert stat.write.inv == True

            yield 2 * (clk.posedge, )

            yield delay(10)
        
            print("DONE")

            raise StopSimulation
        
        return instances()


## Defining an architecture

We define an RTL architecture for the a LatticeSemi ECP5 target, for instance:

In [7]:
from myirl.library.architecture import Architecture
from cyrite.library.targets import ECP5
from yosys.simulator import CXXRTL

class RTLArch(Architecture):
	def __init__(self):
		self.sim_class = CXXRTL
		self.target_class = ECP5

Then we create a design instance and pass this architecture as target:

In [8]:
m = SimMMR("mmr", RTLArch())
tb = m.testbench()
tb.debug()

# tb.design.display_rtl()

DEBUG: Dummysignal `s_9558` : False -> False
DEBUG: Dummysignal `s_9558` : False -> False
 Declare obj 'mmr_decode' in context '(SimMMR 'mmr')'(<class '__main__.SimMMR'>) 


Use Cython binding for dict
DEBUG: not handling type <class 'dict'> in co-simulation. Your Cosimulation may not run correctly.
DEBUG: not handling type <class 'myirl.container.container.<locals>.dummy.<locals>._mixin'> in co-simulation. Your Cosimulation may not run correctly.


 Adding module with name `mmr_decode` 

-- Running command `ls; check' --

1. Executing CHECK pass (checking for obvious problems).
Found and reported 0 problems.

-- Running command `hierarchy -check' --

2. Executing HIERARCHY pass (managing design hierarchy).

-- Running command `write_verilog debug.v' --

3. Executing Verilog backend.

3.1. Executing BMUXMAP pass.

3.2. Executing DEMUXMAP pass.


In [7]:
# Turn 'debug' on for simulation output
tb.run(400, debug = True, wavetrace = 'mmr.vcd', recompile = True)

Compiling /tmp/myirl_mmr_12ud_f3m/mmr_decode_2cc0.pyx because it changed.
[1/1] Cythonizing /tmp/myirl_mmr_12ud_f3m/mmr_decode_2cc0.pyx
running build_ext
building 'runtime.mmr_decode_2cc0' extension
creating build/temp.linux-x86_64-3.10/tmp/myirl_mmr_12ud_f3m
gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -fPIC -DCOSIM_NAMESPACE=mmr_decode_2cc0 -Iruntime -I/tmp/myirl_mmr_12ud_f3m/ -I/usr/share/yosys/include/backends/cxxrtl/runtime -I/usr/local/include/python3.10 -c /tmp/myirl_mmr_12ud_f3m/mmr_decode_2cc0.cpp -o build/temp.linux-x86_64-3.10/tmp/myirl_mmr_12ud_f3m/mmr_decode_2cc0.o
gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -fPIC -DCOSIM_NAMESPACE=mmr_decode_2cc0 -Iruntime -I/tmp/myirl_mmr_12ud_f3m/ -I/usr/share/yosys/include/backends/cxxrtl/runtime -I/usr/local/include/python3.10 -c /tmp/myirl_mmr_12ud_f3m/mmr_decode_2cc0_rtl.cpp -o build/temp.linux-x86_64-3.10/tmp/myirl_mmr_12ud_f3m/mmr_decode_2cc0_rtl.o
g++ -pthread -shar

Tolerate exception: ('Module with name `mmr_decode` already existing', 'While converting `[Instance mmr_decode I/F: [// ID: mmr_decode_0 to mmr_decode]]`')
Using '/tmp/myirl_mmr_12ud_f3m/' for output
Cosimulation: debug not connected to backend
STOP SIMULATION @88


## Waveform display

The `*.vcd` format hides the `MMRPort` record members from the trace. Therefore we need a few monitoring auxiliary signals.

If we change signal names, we will also have to change the config below.
In order to retrieve the signal names generated by the VCD trace, set the `cfg` parameter to None. This will then display all traced signals.

In [8]:
from cyrite import waveutils

In [9]:
config = {
    '.clk' : None,
    '.reset' : None,
    '.ctrl_select_sel_w' : None,
    '.data_in' : None,
    '.data_out' : None,
    '.addr' : None
}

waveutils.draw_wavetrace(tb, 'mmr.vcd', 'clk', cfg = config)

## HDL translation

When a MMR decoder instance is created, the interface, i.e. port and parameters (VHDL generics) is generated from the  Register description as well. The `ctrl` and `stat` signal containers are normally wired up to the peripheral modules, the bus signals map into a specific MMR address space of the CPU.

We build the above design again and explicitely output to VHDL.

In [10]:
from cyrite.simulation import icarus
design = SimMMR("test", icarus.ICARUS)
# Create an instance of the top level tb:
tb = design.testbench()
files = design.elab(targets.VHDL)

 Declare obj 'testbench' in context '(SimMMR 'test')'(<class '__main__.SimMMR'>) 
 Declare obj 'mmr_decode' in context '(SimMMR 'test')'(<class '__main__.SimMMR'>) 
 Writing 'mmr_decode' to file /tmp/myirl_test_69_kxwnt/mmr_decode.vhdl 
 Writing 'testbench' to file /tmp/myirl_test_69_kxwnt/testbench.vhdl 
 Creating library file module_defs.vhdl 


Use Cython binding for dict


Inspect the MMR decoder:

In [11]:
!cat {files[0]}

-- File generated from source:
--     <not available (cython)>
-- (c) 2016-2022 section5.ch
-- Modifications may be lost, edit the source file instead.

library IEEE;
use IEEE.std_logic_1164.all;
use IEEE.numeric_std.all;

library work;

use work.module_defs.all;
use work.txt_util.all;
use work.myirl_conversion.all;

entity mmr_decode is
    port (
        clk : in std_ulogic;
        reset : in std_ulogic;
        addr : in unsigned(11 downto 0);
        wr : in std_ulogic;
        data_in : in unsigned(15 downto 0);
        data_out : out unsigned(15 downto 0);
        stat_read : in t_stat_read;
        stat_write : out t_stat_write;
        ctrl_read : in t_ctrl_read;
        ctrl_write : out t_ctrl_write;
        ctrl_select : out t_ctrl_sel;
        TXD_read : in t_TXD_read;
        TXD_write : out t_TXD_write;
        TXD_select : out t_TXD_sel;
        RXD_read : in t_RXD_read;
        RXD_write : out t_RXD_write;
        RXD_select : out t_RXD_sel
    );
end entity mmr_decode;

We note:

* The interface is dynamically generated, according to the Peripheral map
* The particular VHDL output uses records that are generated from the Register map description

## Synthesis details

To check the details of a particular sequence of mapping steps done inside yosys, we derive the `MMRDesign` again and augment it by a synthesis method.

The `target.map` call actually maps the elaborated design to the target architecture. Then, we call 'stat' on the design via the yosys library:

In [12]:
class SynMMR(MMRDesign):
    def synthesis_stats(self):
        p = MMRPort(self)
        interface = self.generate_interface(self.regdesc)
        inst = self.build(p, interface)
        target = self.target_class("mmr")
        d = inst.elab(target)
        target.map(capture = False)
        design = d[0]
        out = design.run("stat", capture = None)
        return out


Instance the `SynMMR` class and call its particular synthesis method:

In [13]:
m = SynMMR("syn", RTLArch())
out = m.synthesis_stats()

DEBUG: Dummysignal `s_60ef` : False -> False
DEBUG: Dummysignal `s_60ef` : False -> False
 Declare obj 'mmr_decode' in context '(SynMMR 'syn')'(<class '__main__.SynMMR'>) 
 Adding module with name `mmr_decode` 
 Mapping to ECP5 technology 


Use Cython binding for dict


The stat command output finally can be obtained separately through the capture:

In [14]:
print(out)


-- Running command `stat' --

34. Printing statistics.

=== mmr_decode ===

   Number of wires:                 61
   Number of wire bits:            274
   Number of public wires:          24
   Number of public wire bits:     162
   Number of memories:               0
   Number of memory bits:            0
   Number of processes:              0
   Number of cells:                166
     $_AND_                         30
     $_NOT_                         13
     $_OR_                          57
     TRELLIS_FF                     66




We can see the number of (virtual) primitives used by this logic past the mapping stage.